In [1]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
import category_encoders as ce #pip install category_encoders

In [2]:
train = pd.read_csv('train.csv')
train.head(10)

,id,keyword,location,text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,1
1,4,NaN,NaN,forest fire near la rong sask canada,1
2,5,NaN,NaN,al resid ask to shelter in place are be notifi...,1
3,6,NaN,NaN,number peopl receiv wildfir evacu order in cal...,1
4,7,NaN,NaN,just got se not thi photo from rubi alaska as ...,1
5,8,NaN,NaN,rockyfir updat california hwi number close in ...,1
6,10,NaN,NaN,flood disast heavi rain caus flash flood of st...,1
7,13,NaN,NaN,i am on top of the hil and i can see a fire in...,1
8,14,NaN,NaN,there is an emerg evacu happen now in the buil...,1
9,15,NaN,NaN,i am afraid that the tornado is come to our area,1


In [3]:
pos_col_text = train.columns.get_loc('text')+1
train.insert(loc=pos_col_text, column='len_text', value=train['text'].iloc[:].str.len())
train.head(10)

,id,keyword,location,text,len_text,target
0,1,NaN,NaN,our deed are the reason of thi earthquak may a...,63,1
1,4,NaN,NaN,forest fire near la rong sask canada,37,1
2,5,NaN,NaN,al resid ask to shelter in place are be notifi...,108,1
3,6,NaN,NaN,number peopl receiv wildfir evacu order in cal...,54,1
4,7,NaN,NaN,just got se not thi photo from rubi alaska as ...,84,1
5,8,NaN,NaN,rockyfir updat california hwi number close in ...,96,1
6,10,NaN,NaN,flood disast heavi rain caus flash flood of st...,81,1
7,13,NaN,NaN,i am on top of the hil and i can see a fire in...,56,1
8,14,NaN,NaN,there is an emerg evacu happen now in the buil...,66,1
9,15,NaN,NaN,i am afraid that the tornado is come to our area,49,1


In [4]:
ce_bin_keyword = ce.BinaryEncoder(cols = ['keyword'])
keyword_encoded = ce_bin_keyword.fit_transform(train['keyword'])

In [5]:
keyword_encoded

,keyword_0,keyword_1,keyword_2,keyword_3,keyword_4,keyword_5,keyword_6,keyword_7,keyword_8
0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,1
7609,0,0,0,0,0,0,0,0,1
7610,0,0,0,0,0,0,0,0,1
7611,0,0,0,0,0,0,0,0,1


In [6]:
ce_bin_location = ce.BinaryEncoder(cols = ['location'])
location_encoded = ce_bin_location.fit_transform(train['location'])

In [7]:
location_encoded

,location_0,location_1,location_2,location_3,location_4,location_5,location_6,location_7,location_8,location_9,location_10,location_11,location_12
0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0,0,0,0,0,0,0,0,0,0,0,0,1
7609,0,0,0,0,0,0,0,0,0,0,0,0,1
7610,0,0,0,0,0,0,0,0,0,0,0,0,1
7611,0,0,0,0,0,0,0,0,0,0,0,0,1


In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk #pip install nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ben/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
tf_idf_vec = TfidfVectorizer(use_idf=True, smooth_idf=False, ngram_range=(1,1), stop_words='english', tokenizer=nltk.word_tokenize)
tf_idf_data = tf_idf_vec.fit_transform(train['text'])

In [10]:
col_names = tf_idf_vec.get_feature_names()
col_names

['ab',
 'aba',
 'abandon',
 'abandonedp',
 'abbandon',
 'abbott',
 'abbruchsimul',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcnew',
 'abcnorio',
 'abe',
 'aberdeen',
 'aberdeenfanpag',
 'aberdeenfc',
 'aberystwyth',
 'abha',
 'abia',
 'abil',
 'abject',
 'abl',
 'ablaz',
 'ableg',
 'abninfvet',
 'aboard',
 'abomb',
 'abomin',
 'abort',
 'abov',
 'abq',
 'abrancabalero',
 'absenc',
 'absolut',
 'absolutsumya',
 'abstorm',
 'abstract',
 'absurd',
 'absurdli',
 'abubara',
 'abus',
 'abysmaljoin',
 'ac',
 'acaciapen',
 'academia',
 'acarewornheart',
 'acc',
 'accept',
 'access',
 'accid',
 'accidentali',
 'accidentalpropheci',
 'accionempresa',
 'accompani',
 'accord',
 'accordingli',
 'accou',
 'account',
 'accuraci',
 'accus',
 'accustom',
 'acd',
 'acdelco',
 'ace',
 'acebab',
 'acebreakingnew',
 'acenewsdesk',
 'aceorg',
 'acess',
 'ach',
 'achedin',
 'achiev',
 'achievem',
 'achimota',
 'acid',
 'acmilan',
 'acn',
 'acoust',
 'acousticmaloley',
 'acquiesc',
 'acquir',
 'acquisit',
 

In [11]:
tf_idf_array = tf_idf_data.toarray()
text_encoded = pd.DataFrame(tf_idf_array, columns=col_names)

In [12]:
text_encoded

,ab,aba,abandon,abandonedp,abbandon,abbott,abbruchsimul,abbswinston,abbyairshow,abc,...,zonewolf,zoom,zotar,zouma,zouryart,zrnf,zss,zumiez,zurich,zxatheti
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
aux = keyword_encoded.join(other=location_encoded, how='inner')
aux = text_encoded.join(other=aux, how='inner')
train_set = aux.join(other=train[['id', 'len_text', 'target']], how='inner', lsuffix='_')

In [14]:
train_set.head(10)

,ab,aba,abandon,abandonedp,abbandon,abbott,abbruchsimul,abbswinston,abbyairshow,abc,...,location_6,location_7,location_8,location_9,location_10,location_11,location_12,id,len_text,target
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,1,63,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,4,37,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,5,108,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,6,54,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,7,84,1
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,8,96,1
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,10,81,1
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,13,56,1
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,14,66,1
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,15,49,1


### XGBoost

In [16]:
from sklearn.model_selection import train_test_split

X, y = train_set.iloc[:,:-1],train_set.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [19]:
xg_reg = xgb.XGBClassifier(objective ='reg:logistic', subsample=0.3,
                colsample_bytree = 0.6, learning_rate = 0.01,
                max_depth = 1000, alpha = 10, n_estimators = 20)
xg_reg.fit(X_train,y_train)
preds = xg_reg.predict(X_test)
acc = np.sqrt(accuracy_score(y_test, preds))
print("Accuracy Score: %f" % (acc))

Accuracy Score: 0.794574


### Test

In [20]:
test_set = pd.read_csv('test.csv')

In [21]:
pos_col_text = test_set.columns.get_loc('text')+1
test_set.insert(loc=pos_col_text, column='len_text', value=test_set['text'].iloc[:].str.len())
test_set.head(10)

,id,keyword,location,text,len_text
0,0,NaN,NaN,just happen a teribl car crash,31
1,2,NaN,NaN,heard about earthquak is differ not citi stay ...,59
2,3,NaN,NaN,there is a forest fire at spot pond gees are f...,89
3,9,NaN,NaN,apocalyps light spokan wildfir,31
4,11,NaN,NaN,typhoon soudelor kil number in china and taiwan,48
5,12,NaN,NaN,we are shake it is an earthquak,32
6,21,NaN,NaN,they would probabl stil show more life than ar...,70
7,22,NaN,NaN,hey how are you,16
8,27,NaN,NaN,what a nice hat,16
9,29,NaN,NaN,fuck off,9


In [22]:
tf_idf_data = tf_idf_vec.fit_transform(test_set['text'])

In [23]:
col_names = tf_idf_vec.get_feature_names()

In [24]:
tf_idf_array = tf_idf_data.toarray()

In [25]:
tf_idf_df = pd.DataFrame(tf_idf_array, columns=col_names)
test_set_encoded = test_set.join(other=tf_idf_df, how='inner', lsuffix="_")

In [26]:
col_names.append('id_')
col_names.append('len_text')

test_set = test_set_encoded[col_names]

test_set.head(10)

,ab,aba,abandon,abba,abbog,abbott,abbss,abbswinston,abbydphilip,abc,...,zjwhitman,zombi,zone,zoui,zouma,zuma,zvch,zx,id_,len_text
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,31
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,59
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,89
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,31
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,48
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,32
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21,70
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22,16
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,27,16
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,29,9


In [29]:
preds = xg_reg.predict(test_set)

ValueError: feature_names mismatch: ['ab', 'aba', 'abandon', 'abandonedp', 'abbandon', 'abbott', 'abbruchsimul', 'abbswinston', 'abbyairshow', 'abc', 'abcnew', 'abcnorio', 'abe', 'aberdeen', 'aberdeenfanpag', 'aberdeenfc', 'aberystwyth', 'abha', 'abia', 'abil', 'abject', 'abl', 'ablaz', 'ableg', 'abninfvet', 'aboard', 'abomb', 'abomin', 'abort', 'abov', 'abq', 'abrancabalero', 'absenc', 'absolut', 'absolutsumya', 'abstorm', 'abstract', 'absurd', 'absurdli', 'abubara', 'abus', 'abysmaljoin', 'ac', 'acaciapen', 'academia', 'acarewornheart', 'acc', 'accept', 'access', 'accid', 'accidentali', 'accidentalpropheci', 'accionempresa', 'accompani', 'accord', 'accordingli', 'accou', 'account', 'accuraci', 'accus', 'accustom', 'acd', 'acdelco', 'ace', 'acebab', 'acebreakingnew', 'acenewsdesk', 'aceorg', 'acess', 'ach', 'achedin', 'achiev', 'achievem', 'achimota', 'acid', 'acmilan', 'acn', 'acoust', 'acousticmaloley', 'acquiesc', 'acquir', 'acquisit', 'acr', 'acronym', 'acryl', 'act', 'actavi', 'a